In [5]:
#펭수/워크맨 csv 불러와서 시각화 및 머신러닝
# 패키지 import
import datetime as dt
import pandas as pd   #df 분석용
import requests
import time
import urllib.request #
import re     
import konlpy

from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from nltk import FreqDist

# 기계 학습
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
############### Read #################
peng_intro = pd.read_csv('peng_intro.csv')  # 펭수의 intro DF 
work_intro = pd.read_csv('work_intro.csv')  # 워크맨의 intro DF 

peng = pd.read_csv('peng.csv')               #펭수의 동영상list DF
peng_reply = pd.read_csv('peng_reply.csv')   #펭수의 동영상별 댓글list DF

work = pd.read_csv('work.csv')               #워크맨의 동영상list DF
work_reply = pd.read_csv('work_reply.csv')   #워크맨의 동영상별 댓글list DF

############# 구분자 추가 ################
for i in range(len(peng)): peng['구분'] = '펭수' 
for i in range(len(work)): work['구분'] = '워크맨'
for i in range(len(peng_reply)): peng_reply['구분'] = '펭수'    
for i in range(len(work_reply)): work_reply['구분'] = '워크맨'
    
############## DF 통합 ##################
# 1) 펭수 + 워크맨 intro DF 통합
intro_paw = pd.DataFrame(peng_intro)       
intro_paw = intro_paw.append(work_intro)

# 2) 펭수 + 워크맨 동영상list DF 통합
paw = pd.DataFrame(peng)     
paw = paw.append(work)

# 3) 펭수 + 워크맨 동영상별 댓글list DF 통합
paw_reply = pd.DataFrame(peng_reply)  
paw_reply = paw_reply.append(work_reply)    


In [17]:
# csv 파일의 columnms 확인
paw.columns
paw_reply.columns

Index(['Unnamed: 0', 'ID', 'Comment', 'Like', 'Title_number', '구분'], dtype='object')

In [18]:
# csv 파일에서 필요한 column만 리스트로 저장
paw_title_list = paw.제목.tolist()
paw_content_list = paw.본문내용.tolist()
paw_reply_list = paw_reply.Comment.tolist()

In [19]:
# 리스트 출력하여 확인
print(paw_title_list[:3])  #부하로 인해 테스트로 3개만 출력

['[Ep 67] 전 세계 게 섰거라 (feat. 외교부)', '[Ep.66]  화보 모델 펭수', '펭수와 팬들의 최강 컬래버레이션 신상뮤비 (feat. 챌린지♡)']


In [20]:
# 형태소 분석
from konlpy.tag import Okt
okt=Okt()

In [21]:
# 댓글 내용 명사 분석
reply_list=[]
for i in range(len(paw_reply_list)):
    reply_list.append(okt.nouns(paw_reply_list[i]))

In [22]:
# 리스트 안의 리스트 하나의 리스트로 만들기
def flatten (n):
    org =[]
    for i in n :
        if (isinstance(i,list)):
            org += flatten(i)
        else:
            org.append(i)
    return org

In [23]:
# 댓글 내용 하나의 리스트로 취합
reply_f = flatten(reply_list)

In [24]:
###############  NMF #################
reply_f

['펭수',
 '곧',
 '백만이',
 '짜싁',
 '날씨',
 '춥다',
 '감기',
 '조심',
 '허구',
 '국밥',
 '악플러',
 '빚',
 '무슨',
 '교육방송',
 '예능',
 '프로그램',
 '더',
 '자막',
 '편집',
 '프로그램',
 '포텐',
 '중',
 '물품',
 '가지',
 '제',
 '자신',
 '초반',
 '영상',
 '온',
 '사람',
 '죄',
 '댓글',
 '지도',
 '방송국',
 '방송',
 '국가',
 '더',
 '펭수',
 '제작',
 '진',
 '여러분',
 '펭러',
 '뷰',
 '펭수',
 '골드',
 '버튼',
 '언박싱',
 '사장',
 '방',
 '하쟈',
 '외교',
 '뷰',
 '살',
 '다산',
 '상징',
 '뇨',
 '외교부',
 '편이',
 '진짜',
 '레전드',
 '어른',
 '합',
 '꿀벌',
 '펭수',
 '판다',
 '펭수',
 '취권',
 '펭수',
 '등등',
 '개',
 '인기',
 '꿀',
 '펭수',
 '나',
 '덴마크',
 '한국',
 '사람',
 '여기',
 '친구',
 '너',
 '동영상',
 '전도',
 '영어',
 '자막',
 '영어',
 '자막',
 '회사',
 '사람',
 '펭수',
 '얘기',
 '탈',
 '뭐',
 '펭수',
 '마트료시카',
 '굿',
 '구매',
 '분',
 '무한도전',
 '시어머니',
 '팬',
 '때문',
 '곤혹',
 '거',
 '펭수',
 '사람',
 '하자',
 '인간',
 '펭수',
 '주접',
 '댓글',
 '악플',
 '주접',
 '댓글',
 '펭수',
 '펭수',
 '항상',
 '반말',
 '짓거리',
 '남',
 '무시',
 '말투',
 '다나',
 '를',
 '엄마',
 '화가',
 '왜',
 '펭',
 '말투',
 '왜',
 '그로',
 '닷',
 '말투',
 '고맙슴돠',
 '펭수',
 '펭',
 '팬',
 '분',
 '듯',
 '해',
 '펭수',
 '팬',
 '위로',
 '주지',
 '팬',
 '끼리'

In [26]:
# sklearn의 CountVectorizer 함수 사용, bow(bag of words)로 변환하는 객체 생성
bow_transformer = CountVectorizer().fit(reply_f)

In [27]:
len(bow_transformer.vocabulary_)

5177

In [28]:
# reply_f 변형
reply_f = bow_transformer.transform(reply_f)

In [29]:
# reply_f 확인
print('Shape of Sparse Matrix: ', reply_f.shape)
print('Amount of Non-Zero occrrences: '), reply_f.nnz

Shape of Sparse Matrix:  (27879, 5177)
Amount of Non-Zero occrrences: 


(None, 20457)

In [30]:
# reply_f는 현재 document-term matrix로 구성됨.
# NMF를 적용하기 위해서는 term-document matrix가 필요.
# 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.
type(reply_f)

scipy.sparse.csr.csr_matrix

In [31]:
# compressed sparse row matrix인 reply_f를 transpose 한다.
reply_f_trans = reply_f.transpose()
type(reply_f_trans)

scipy.sparse.csc.csc_matrix

In [32]:
print('Shape of Sparse Matrix: ', reply_f_trans.shape)
print('Amount of Non-Zero occrrences: '), reply_f_trans.nnz

Shape of Sparse Matrix:  (5177, 27879)
Amount of Non-Zero occrrences: 


(None, 20457)

In [33]:
# NMF 모델 객체 생성
from sklearn.decomposition import NMF
model = NMF(n_components=2)

In [34]:
# NMF 모델 학습
W = model.fit_transform(reply_f_trans)

In [35]:
H = model.components_

In [36]:
type(H)

numpy.ndarray

In [37]:
H.shape

(2, 27879)

In [38]:
# 클러스터링 결과 점수
print(H)

[[0.15668896 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [39]:
# 전체 문서 별 best 점수에 해당하는 클러스터 label
pred_labels = H.argmax(axis=0)
len(pred_labels)

27879

In [40]:
pred_labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [41]:
# 실제 labels이..? 없.....?
data_y

NameError: name 'data_y' is not defined